This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 63.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 33.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [2]:
import os
import pandas as pd

os.environ["OPENAI_API_KEY"] = "voc-6133227481266773645905672e78f1963380.60536949"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
import openai
from io import StringIO


In [13]:
# load the model
# Environment variables

MODEL_NAME = 'gpt-3.5-turbo'

# Set your API key
# openai_api_key = 'your-api-key-here'

# Create an instance of the OpenAI LLM
llm = OpenAI(model_name=MODEL_NAME, temperature=0)

INSTRUCTION = "Generate a CSV file with at least 20 real estate listings."
SAMPLE_LISTING = \
"""
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


# Generate real estate listings

In [14]:
# Define the user prompt for generating real estate listings
user_prompt = '''
Generate a comprehensive CSV file with at least 20 distinct real estate listings. Each listing should be meticulously organized into columns, showcasing the following attributes:

  - neighborhood: [Name]
  - price: [Price]
  - bedrooms: [Number of bedrooms]
  - bathrooms: [Number of bathrooms]
  - house_size: [Size in sqft]
  - Description: [Descriptions of the property, highlight features such as design, energy efficiency, upgrades, and the surrounding environment.]
  - Neighborhood Description: [Brief description of the neighborhood, including schools, community, nearby amenities, and transportation options.]

Example Entry Format:
Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
Green Oaks,"$800,000",3,2,"2,000 sqft","Nestled in Green Oaks, this eco-friendly haven features a 3-bedroom, 2-bathroom layout with solar panels and efficient insulation. Highlights include abundant natural light, hardwood floors, and an open-concept kitchen that leads to a lush backyard, embodying a sanctuary for eco-conscious living.", "The neighborhood of Green Oaks is celebrated for its vibrant and environmentally-aware community, boasting organic stores, community gardens, and convenient transit options, rendering it perfect for those prioritizing sustainability and community engagement."

Structure the CSV with clear headers for each column. Follow the example provided to format each subsequent row with information specific to a different property listing. Make sure you generate 20 unique listings.
'''

# Generate the listings using OpenAI API
messages = [{"role": "user", "content": user_prompt}]
response = openai.ChatCompletion.create(
    model=MODEL_NAME,
    messages=messages,
    temperature=0.5,  # Adjust for creativity level
    max_tokens=4000,  # Increased max tokens to ensure full response
)

# Extract the generated listings
listings = response.choices[0].message['content']
print("Generated Listings:\n", listings)  # Debugging information

# Clean and validate the generated CSV data
def clean_csv_data(csv_data):
    lines = csv_data.split('\n')
    cleaned_lines = []
    for line in lines:
        if line.count('"') % 2 != 0:  # Check for unclosed quotes
            line += '"'  # Add closing quote if missing
        cleaned_lines.append(line)
    return '\n'.join(cleaned_lines)

cleaned_listings = clean_csv_data(listings)

# Use StringIO to read the string as a file
csv_data = StringIO(cleaned_listings)
try:
    df = pd.read_csv(csv_data)
except pd.errors.ParserError as e:
    print("Error parsing CSV data:", e)
    print("Cleaned Listings:\n", cleaned_listings)  # Print cleaned data for debugging

# Display the DataFrame to verify
print(df)

# Save the DataFrame to a CSV file
df.to_csv('house_listings.csv', index=False)

print("CSV file 'house_listings.csv' has been generated and saved.")

Generated Listings:
 Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
Downtown Loft,"$500,000",1,1,"800 sqft","This modern loft in the heart of downtown offers an open floor plan with exposed brick walls, high ceilings, and large windows providing stunning city views. The building features a rooftop terrace and is steps away from trendy restaurants and shops.","Located in the bustling downtown area, this neighborhood is known for its vibrant nightlife, cultural events, and easy access to public transportation, making it ideal for urban dwellers."
Seaside Villa,"$1,200,000",4,3,"3,500 sqft","This luxurious villa by the sea boasts 4 bedrooms, 3 bathrooms, a gourmet kitchen, and a private pool overlooking the ocean. The property features high-end finishes, a spacious deck for outdoor entertaining, and breathtaking sunset views.","Nestled in a charming seaside community, this neighborhood offers a relaxed coastal lifestyle with access to pristine beache

In [16]:
df.head()

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Downtown Loft,"$500,000",1,1.0,800 sqft,This modern loft in the heart of downtown offe...,"Located in the bustling downtown area, this ne..."
1,Seaside Villa,"$1,200,000",4,3.0,"3,500 sqft",This luxurious villa by the sea boasts 4 bedro...,"Nestled in a charming seaside community, this ..."
2,Mountain Retreat,"$700,000",3,2.5,"2,200 sqft",Escape to this serene mountain retreat featuri...,"Set in a peaceful mountain town, this neighbor..."
3,Historic Brownstone,"$900,000",5,4.0,"3,000 sqft",Step back in time with this historic brownston...,"Located in a historic district, this neighborh..."
4,Lakefront Cottage,"$600,000",2,1.5,"1,500 sqft",Enjoy lakeside living in this charming cottage...,"Situated on the shores of a tranquil lake, thi..."


# Store Listings in a Vector Database
### Vector Database Setup: Initialize and configure ChromaDB to store real estate listings.

### Generate and Store Embeddings: Convert the LLM-generated listings into embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.


In [17]:
# Initialize and configure ChromaDB to store real estate listings

listings_csv = CSVLoader(file_path='./house_listings.csv')
docs = listings_csv.load()

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
# Prepare the DB.
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [18]:
print(dir(db))

['_Chroma__query_collection', '_LANGCHAIN_DEFAULT_COLLECTION_NAME', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_client', '_client_settings', '_collection', '_cosine_relevance_score_fn', '_embedding_function', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_persist_directory', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_texts', 'afrom_documents', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector', 'as_retriever', 'asearch', 'asimilarity_search', 'asim

# Build the User Preference Interface

In [20]:
query = "A quiet four-bedroom house with a spacious kitchen in a convenient neighborhood."

PROMPT_TEMPLATE = """
Answer the question based on the context below:

{context}
----------------------------------------------------------
Answer the question {question} based on the above context, provide simple explanations towards specified preference.
"""

# Search the DB.
results = db.similarity_search_with_relevance_scores(query, k=3)
if len(results) == 0 or results[0][1] < 0.8:
    print(f"Unable to find matching results.")
else:
    context = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context, question=query)
    print(f"Generated Prompt:\n{prompt}")

    model = ChatOpenAI()
    response = model.predict(prompt)

    formatted_response = f"Response: {response}"
    print(formatted_response)

Generated Prompt:
Human: 
Answer the question based on the context below:

Neighborhood: Family Home
Price: $650,000
Bedrooms: 4
Bathrooms: 2.5
House Size: 2,300 sqft
Description: This spacious family home features 4 bedrooms, 2.5 bathrooms, a bonus room, and a backyard with a playset and garden. The property is located in a family-friendly neighborhood with top-rated schools, parks, and community events.
Neighborhood Description: Situated in a family-friendly community, this neighborhood offers a safe and welcoming environment for families with children, with tree-lined streets, playgrounds, and community centers, fostering a sense of belonging and community spirit.

---

Neighborhood: Countryside Farmhouse
Price: $750,000
Bedrooms: 5
Bathrooms: 3.0
House Size: 2,500 sqft
Description: Experience country living in this charming farmhouse with 5 bedrooms, 3 bathrooms, a wrap-around porch, and a barn on expansive acreage. The property is surrounded by rolling hills and farmland, perfect 

# Generate Augmented Response

In [24]:
# Generate personalized response
def generate_personalized_response(listing_details, buyer_preferences):
    
    
    response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a real estate agent.For each retrieved listing, tailor it to resonate with the buyers' specific preferences. This invovles subtly emphasizing aspects of the property that align with what the buyer is looking for"},
                    {"role": "user", "content": buyer_preferences}
                ],
                temperature=0.5,  # Adjust for creativity level
                max_tokens=1500,
    )
    return response.choices[0].message.content

In [25]:
listing_details = {
    'description': 'This spacious family home features 4 bedrooms, 2.5 bathrooms, a bonus room, and a backyard with a playset and garden.',
    'neighborhood_description': 'The property is located in a family-friendly neighborhood with top-rated schools, parks, and community events.',
    'price': '$650,000',
    'bedrooms': '4',
    'bathrooms': '2.5',
    'house_size': '2300'
}

buyer_preferences = "A quiet four-bedroom house with a spacious kitchen in a convenient neighborhood."

# Generate the personalized description
personalized_response = generate_personalized_response(listing_details, buyer_preferences)

print(personalized_response)

Welcome to this lovely four-bedroom home in a peaceful neighborhood, perfect for those seeking a tranquil living environment. The spacious kitchen is ideal for those who love to cook and entertain, with ample room for preparing delicious meals and hosting gatherings. Located in a convenient neighborhood, this property offers easy access to amenities, making it a great choice for those looking for a comfortable and practical living space.
